# Discovering Defining Equations with Test-Driven Development

Test-Driven Development (TDD) is more than just a way of testing code. The proponents of TDD cite many benefits to the practice of writing tests first, **then** writing the code:

* The tests serve as a specification for what the code that will be written should do
* Developers are able to understand what the program should do before trying to write it
* Developers can use the tests to help design the code that will pass them
* Developers aren't afraid to change code as needed, because they know the modified code will be thoroughly tested

I want to focus on the third benefit above. We'll use the tests to help us write the code we want.

## Multiplication by Repeated Addition

Let's write a function `(mul a b)` that returns `a*b` when `a` and `b` are natural numbers. Of course, we could just use the built-in function `*`, but we are learning how to write recursive functions, so let's write the function ourselves.

We are using TDD, so let's start by thinking of some unit tests:

    (mul 3 10) = 30
    (mul 5 19) = 95
    (mul 2 10) = 20

Now, how should we write this? We are using "repeated addition" and going from first principles (no built-in `*`, remember?) so we have to dig deep and remember what multiplication really is. So we let ourselves go back to 3rd grade and remember that $5 \times 19 = \underbrace{19 + 19 + \cdots + 19}_{5\text{ times}}$. So maybe, we should rewrite our unit tests this way:

    (mul 3 10) = 10 + 10 + 10
    (mul 5 19) = 15 + 15 + 15 + 15 + 15
    (mul 2 10) = 10 + 10

Now, this is the critical moment. What we want to do is to consider the unit tests for a problem and for its "next smaller" problem. It takes some practice to find the "next smaller", but for natural numbers, the next smaller problem to `n` is usually `n-1`. So let's try that. We see these two unit tests:

    (mul 3 10) = 10 + 10 + 10
    (mul 2 10) = 10 + 10

If we only had the first unit test, but not the second, I would have proceeded by **adding that second unit test**. That's the TDD approach: We want to make sure that the function returns the right value for some set of inputs and for **any potential set of inputs that may appear in a recursive call** of that original test. Here, we're betting that `(mul 2 10)` is likely to be a recursive call of `(mul 3 10)`, because we are guessing that the "next smaller" value of `n` will be `n-1`, which it usually is.

So let's look at the two unit tests, this time an eye to answering this question:

> How can we use the answer to `(mul 2 10)` to find the answer to `(mul 3 10)`?

In other words, how can we find the answer to the `n` problem by using the answer to the "next smaller" value of `n`? Looking at the unit tests, we can conjecture that 

    (mul 3 10) = 10 + 10 + 10
    (mul 2 10) = 10 + 10
    ----------------------------
    (mul 3 10) = (mul 2 10) + 10

We're almost there! What's left is just to generalize this specific equation to a defining equation. I.e., not just `3` and `10`, but any `a` and `b`:

    (mul a b) = (mul a-1 b) + b

This looks promising, so let's write it in ACL2 notation:

    (mul a b) = (+ (mul (- a 1) b) 
                   b)

This is not the whole story, of course. We need another defining equation to stop the "loop". Let's do the obvious:

    (mul a b) = 0,                        if a=0 or anything other than a positive integer
    (mul a b) = (+ (mul (- a 1) b) 
                   b),                    otherwise

Now we are ready to translate this to ACL2!

In [ ]:
(defsnapshot mul-definition)

(definec mul (a :nat b :nat) :nat
  (if (zp a)
      0
      (+ (mul (- a 1) b)
         b)))

(check-expect (mul 3 10) 30)
(check-expect (mul 5 19) 95)
(check-expect (mul 2 10) 20)


Excellent! All of the unit tests passed. This isn't always true. Oh let's be honest, it's almost never true! But it does show how the TDD approach can help you write correct code.

To take this a step farther, you can think of some randomized tests we could perform. One that comes to mind is that `mul` should be commutative:

    (test? (equal (mul a b) 
                  (mul b a)))

Come to think of it, `mul` should be associative, too:

    (test? (equal (mul (mul a b) c)
                  (mul a (mul b c))))

And really, since we have a built-in that does multiplication, what we really want to verify is that `mul` is the same as `*`:

    (test? (equal (mul a b) (* a b))

Let's try that:

In [ ]:
(defsnapshot mul-definition)

(definec mul (a :nat b :nat) :nat
  (if (zp a)
      0
      (+ (mul (- a 1) b)
         b)))

(check-expect (mul 3 10) 30)
(check-expect (mul 5 19) 95)
(check-expect (mul 2 10) 20)

(test? (equal (mul a b) 
              (mul b a)))

(test? (equal (mul (mul a b) c)
              (mul a (mul b c))))

(test? (equal (mul a b) (* a b)))

The results are disappointing. First, it's clear that ACL2 is choosing random values for `a` and `b` that are not what we expect. We meant for `a` and `b` to be natural numbers, but ACL2 is choosing values like `61/85`. To fix this, we need to place a `natp` hypothesis in the property. The more concerning issue is that ACL2 found counterexamples to the last test! It appears that `(mul a b)` is not always the same thing as `(* a b)`. Looking at the values of `a` and `b` that ACL2 claims are counterexamples, it again that appears that the problems only happen when ACL2 tries values for `a` and `b` that are not natural numbers. So let's fix the property to apply only to natural numbers and see what happens.

In [ ]:
(defsnapshot mul-definition)

(definec mul (a :nat b :nat) :nat
  (if (zp a)
      0
      (+ (mul (- a 1) b)
         b)))

(check-expect (mul 3 10) 30)
(check-expect (mul 5 19) 95)
(check-expect (mul 2 10) 20)

(test? (implies (and (natp a)
                     (natp b))
                (equal (mul a b) 
                       (mul b a))))

(test? (implies (and (natp a)
                     (natp b))
                (equal (mul (mul a b) c)
                       (mul a (mul b c)))))

(test? (implies (and (natp a)
                     (natp b))
                (equal (mul a b) (* a b))))

Perfect! All unit tests passed, and the random tests all passed.

> Since all the unit tests and random tests passed, you may be tempted now to try using `thm` to see if ACL2 can prove these theorems automatically. Go ahead and try. (Spoilers: It needs help....)

## Logarithms for Natural Numbers

Now let's try a harder function to define. Consider this problem:

> Given a natural number $x$, find the largest natural number $n$ such that $x \le 10^n$

The number $n$ is the best approximation to $\log x$. In fact, $n = \lfloor \log x \rfloor$.

We will write a function that computes this using TDD. At the beginning, we don't have a "real" definition for `natlog`, so we start out with a default definition, where `(natlog x) = x`. We also introduce some tests, for example, we expect that `(natlog 100) = 2` and `(natlog 1000) = 3`. As experienced programmers, we also try out some test cases for numbers around 100 and 1000. At this time, we can also add a randomized test that says our function meets the requirements above: The function returns the largest natural number $n$ such that $x \le 10^n$.

In [ ]:
(defsnapshot natlog-definition)

(definec natlog (x :nat) :nat
    (declare (ignorable x))
    0)

(check-expect (natlog 100) 2)
(check-expect (natlog 1000) 3)
(check-expect (natlog 98) 1)
(check-expect (natlog 102) 2)
(check-expect (natlog 980) 2)
(check-expect (natlog 1020) 3)

(test? (<= (expt 10 (natlog x)) 
           n))
(test? (implies (> n (natlog x))
                (> (expt 10 n) x)))

> You may be wondering: What is this "declare ignorable" business in the definition? When a function parameter is not used in the body of the function, it is being ignored in ACL2 slang. In this case, `x` is ignored, since the function always returns `0`. This is usually a programming error; after all, if the parameter isn't needed, why include it? So ACL2 treats it as an error and reguses to accept the function. In some rare cases, like this one, the programmer's intent **is** to ignore the parameter, so `(declare (ignorable x))` is the way to let the ACL2 compiler know that ignoring `x` is intentional and not an error.

As expected, both tests failed. This is perfectly fine, since we haven't even written the function. The key is to use these test cases to help us **write** the function.

So let's see. We know that

    (natlog 100)  = 2
    (natlog 1000) = 3

This suggests something that agrees with our knowledge of logs. In fact, it suggests this equation:

    (natlog x) = (+ (natlog (/ x 10)) 1)

That is, if we divide $n$ by 10, the log should go down by 1. This seems to work for the cases when `n=1000` and `n=100`. What about the other cases?

    (natlog 1020) = 3
    (natlog 102)  = 2

Yes, it appears we're onto something.

> At this point, you will probably think that I've stacked the deck, since both 1020 and 102 were already in the unit tests. The reality is that only one of those may have been there earlier, but I would have then added the other unit test based on this example!

OK, so it appears that we have a good defining equation:

    (natlog x) = (+ (natlog (/ x 10)) 1)

But it's clearly not enough. We need something to keep us from an infinite loop, since this just keeps repeating with smaller and smaller values of $n$. Let's do the obvious:

    (natlog x) = 0,                        if x=0 or anything other than a positive integer
    (natlog x) = (+ (natlog (/ x 10)) 1),  otherwise

Let's try that!

In [ ]:
(defsnapshot natlog-definition)

(definec natlog (x :nat) :nat
  (if (zp x)
      0
      (+ (natlog (/ x 10)) 1)))

(check-expect (natlog 100) 2)
(check-expect (natlog 1000) 3)
(check-expect (natlog 98) 1)
(check-expect (natlog 102) 2)
(check-expect (natlog 980) 2)
(check-expect (natlog 1020) 3)

(test? (<= (expt 10 (natlog x)) 
           n))
(test? (implies (> m (natlog x))
                (> (expt 10 n) x)))

Disaster! The code doesn't even compile. ACL2 gives us the following error:

    Form:  ( TEST-BODY-CONTRACTS NATLOG... ) 

    **Summary of Cgen/testing**
    We tested 4 examples across 1 subgoals, of which 4 (4 unique) satisfied
    the hypotheses, and found 3 counterexamples and 1 witnesses.

    We falsified the conjecture. Here are counterexamples:
     [found in : "top"]
     -- ((X 83))
     -- ((X 3))
     -- ((X 8))
    
    Cases in which the conjecture is true include:
     [found in : "top"]
     -- ((X 10))
    
    Test? found a counterexample.
    Body contract falsified in: 
     -- (EXTRA-INFO '(:GUARD (:BODY NATLOG)) '(NATLOG (* X (/ 10))))

Start with the first line. This tells us that ACL2 was trying to test the body contracts for the function. Recall that the body contracts say that the body of the function definition satisfies all the contracts for all functions in the definitions, assuming that the function is called with proper arguments. It appears we didn't get far, because ACL2 is saying that it was able to find inputs that caused body contract violations. In particular, the contracts were not satisfied when `x` was 83, 3, or 8, though they were satisfied when `x` was 10. Moreover, the last line tells us that the body contract that failed was in the expression `(NATLOG (* X (/ 10)))`.

I hope you can see the error from that. The body contract for `(natlog x)` is that `x` is a natural number. When `x` is 10, `x/10` is 1, which is also a natural number, so the body contract is satisfied. But when `x=83`, `x/10=8.3` which is **not** a natural number, so the body contract is violated.

What we really meant was $\lfloor x/10 \rfloor$ instead of $x/10$. Luckily, ACL2 has a built-in function to compute floors.  `(floor a b)` is exactly $\lfloor a/b \rfloor$. Let's fix that:

In [ ]:
(defsnapshot natlog-definition)

(definec natlog (x :nat) :nat
  (if (zp x)
      0
      (+ (natlog (floor x 10)) 1)))

(check-expect (natlog 100) 2)
(check-expect (natlog 1000) 3)
(check-expect (natlog 98) 1)
(check-expect (natlog 102) 2)
(check-expect (natlog 980) 2)
(check-expect (natlog 1020) 3)

(test? (<= (expt 10 (natlog x)) 
           n))
(test? (implies (> n (natlog x))
                (> (expt 10 n) x)))

Oh, dear. It still doesn't work. The function didn't pass any of our unit tests! Let's start with this one:

    ACL2S !>>(CHECK-EXPECT (NATLOG 100) 2)
    
    Error in CHECK-EXPECT: Check failed (values not equal).
    Returned value: 3
    Expected value: 2
     :FAILED

I don't see why this failed. According to our equation,

    (NATLOG 100) = 1 + (NATLOG 10)
                 = 1 + 1 + (NATLOG 1)
                 = 1 + 1 + 0

So it should be returning 2. I know it isn't, because ACL2 tells me it returned 3. But it looks like my equation is correct. In this situation, I know what's happening is that the program is doing something other than what I'm expecting. I must be making an incorrect assumption about my program, so I just have to figure out what that is. I can do that by adding more unit tests. In this case, I am assuming that the program is returning the correct value for `(NATLOG 10)` and `(NATLOG 1)`, so let's make sure that it is.

In [ ]:
(defsnapshot natlog-definition)

(definec natlog (x :nat) :nat
  (if (zp x)
      0
      (+ (natlog (floor x 10)) 1)))

(check-expect (natlog 1) 0)
(check-expect (natlog 10) 1)
(check-expect (natlog 100) 2)
(check-expect (natlog 1000) 3)
(check-expect (natlog 98) 1)
(check-expect (natlog 102) 2)
(check-expect (natlog 980) 2)
(check-expect (natlog 1020) 3)

(test? (<= (expt 10 (natlog x)) 
           x))
(test? (implies (> n (natlog x))
                (> (expt 10 n) x)))

I can now see that my assumptions are wrong. In particular,

    ACL2S !>>(CHECK-EXPECT (NATLOG 1) 0)
    
    Error in CHECK-EXPECT: Check failed (values not equal).
    Returned value: 1
    Expected value: 0
     :FAILED

So let's look carefule at this case:

    (NATLOG 1) = 1 + (NATLOG 0)
               = 1 + 0
               = 1

OK, I see why the function returned 1 for this, even though I know the correct value is 0. Looking at the defining equations I used, I can spot my mistake:

    (natlog x) = 0,                        if x=0 or anything other than a positive integer
    (natlog x) = (+ (natlog (/ x 10)) 1),  otherwise

We shouldn't be returning 0 whenb `x=0`. We should have stopped the "loop" sooner, when `x=1`. So the defining equations should have been:

    (natlog x) = 0,                        if x=1 or anything other than a positive integer
    (natlog x) = (+ (natlog (/ x 10)) 1),  otherwise

Unfortunately, there isn't a built-in predicate like `zp` for "if x=1 or anything other than a positive integer", so we will have to write our own:

In [ ]:
(defsnapshot natlog-definition)

(definec natlog (x :nat) :nat
  (if (or (= x 1)
          (not (and (integerp x)
                    (> x 0))))
      0
      (+ (natlog (floor x 10)) 1)))

(check-expect (natlog 1) 0)
(check-expect (natlog 10) 1)
(check-expect (natlog 100) 2)
(check-expect (natlog 1000) 3)
(check-expect (natlog 98) 1)
(check-expect (natlog 102) 2)
(check-expect (natlog 980) 2)
(check-expect (natlog 1020) 3)

(test? (<= (expt 10 (natlog x)) 
           x))
(test? (implies (> n (natlog x))
                (> (expt 10 n) x)))

Well, progress! Now we know that the unit tests passed when `x` is 1, 10, 100, 1000, 102, or 1020. But the unit tests failed when `x` is 98 or 980. You can probably see a pattern here, right? The code fails for numbers that are close to a power-of-10 boundary, but below the boundary. Let's see if we can figure out why by going back to our defining equations:

    (natlog 98) = 1 + (natlog 9)
                = 1 + 1 + (natlog 0)
                = 1 + 1 + 0
                = 2

OK, that's the problem. `(natlog 9)` should be equal to 0, but we're looping one more time. This is similar to the first bug, in that we're not stopping the loop soon enough. Here are our defining equations:

    (natlog x) = 0,                        if x=1 or anything other than a positive integer
    (natlog x) = (+ (natlog (/ x 10)) 1),  otherwise

These stop the loop when `x = 1`. We should have stopped sooner, when `x=10`. Here are the new defining equations:

    (natlog x) = 0,                        if x=10 or anything other than a positive integer
    (natlog x) = (+ (natlog (/ x 10)) 1),  otherwise

And here is the ACL2 code:

In [ ]:
(defsnapshot natlog-definition)

(definec natlog (x :nat) :nat
  (if (or (= x 10)
          (not (and (integerp x)
                    (> x 0))))
      0
      (+ (natlog (floor x 10)) 1)))

(check-expect (natlog 1) 0)
(check-expect (natlog 10) 1)
(check-expect (natlog 100) 2)
(check-expect (natlog 1000) 3)
(check-expect (natlog 98) 1)
(check-expect (natlog 102) 2)
(check-expect (natlog 980) 2)
(check-expect (natlog 1020) 3)

(test? (<= (expt 10 (natlog x)) 
           x))
(test? (implies (> m (natlog x))
                (> (expt 10 n) x)))

It still failed! It still fails when `n=98`. Let's use our defining equations to see what's happening:

    (natlog x) = 0,                        if x=10 or anything other than a positive integer
    (natlog x) = (+ (natlog (/ x 10)) 1),  otherwise

    (natlog 98) = 1 + (natlog 9)
                = 1 + 1 + (natlog 0)
                = 1 + 1 + 0
                = 2

Can you see the problem? We meant to break the loop when $x=10$, but the problem is we went directly to $x=9$. What we really meant was to break the loop whenever $x\le10$.

Wow. Programming is hard, isn't it? This is just demoralizing. At times like this, I have to remind myself that the computer is not the boss of me. I'm the programmer, darn it, and that means that I'm the boss of it. Sure, programming is hard, but if I'm determined (some would say "stubborn") enough, the computer has no chance. I will win this battle.

My spirits are back up, so let's try to fix the defining equations again:

    (natlog x) = 0,                        if x<=10 or anything other than a positive integer
    (natlog x) = (+ (natlog (/ x 10)) 1),  otherwise

Translating this into ACL2 is straightforward:

In [ ]:
(defsnapshot natlog-definition)

(definec natlog (x :nat) :nat
  (if (or (<= x 10)
          (not (and (integerp x)
                    (> x 0))))
      0
      (+ (natlog (floor x 10)) 1)))

(check-expect (natlog 1) 0)
(check-expect (natlog 10) 1)
(check-expect (natlog 100) 2)
(check-expect (natlog 1000) 3)
(check-expect (natlog 98) 1)
(check-expect (natlog 102) 2)
(check-expect (natlog 980) 2)
(check-expect (natlog 1020) 3)

(test? (<= (expt 10 (natlog x)) 
           x))
(test? (implies (> n (natlog x))
                (> (expt 10 n) x)))

Oh, my god, it still FAILs. That's it, I give up. I'm just not good at programming. I knew I should have gone to beauty school instead.

Wait, wait. The computer's not the boss of me. I'm the programmer, so I'm the boss of it. OK, let's see. Breathe. Breathe.

OK, what went wrong? `(NATLOG 1)` passed. But `(NATLOG 10)` failed. It should return 1, but here are our defining equations:

    (natlog x) = 0,                        if x<=10 or anything other than a positive integer
    (natlog x) = (+ (natlog (/ x 10)) 1),  otherwise

Clear as day, when $x=10$ this will return 0. We want to return 0 for $x=9$, and $x=10$ should be the smallest number such that `(natlog x) = 1`. Ah, I see. That shouldn't be `<=`; it should just be `<`:

    (natlog x) = 0,                        if x<10 or anything other than a positive integer
    (natlog x) = (+ (natlog (/ x 10)) 1),  otherwise

Translating this to ACL2 is very easy (in fact, we only have to change one character from our previous version):

In [ ]:
(defsnapshot natlog-definition)

(definec natlog (x :nat) :nat
  (if (or (< x 10)
          (not (and (integerp x)
                    (> x 0))))
      0
      (+ (natlog (floor x 10)) 1)))

(check-expect (natlog 1) 0)
(check-expect (natlog 10) 1)
(check-expect (natlog 100) 2)
(check-expect (natlog 1000) 3)
(check-expect (natlog 98) 1)
(check-expect (natlog 102) 2)
(check-expect (natlog 980) 2)
(check-expect (natlog 1020) 3)

(test? (<= (expt 10 (natlog x)) 
           x))
(test? (implies (> n (natlog x))
                (> (expt 10 n) x)))

Yes, yes, yes! All the unit tests passed!

That feeling you're feeling right now is called "Programmer's High". It's the reason most of us who have been programmers for years and years still do it.

Unfortunately, something is still wrong. ACL2 is finding counterexamples for our properties. Looking at the counterexamples, we see that the property is true when `X` is 47, 28, or 10,478. But the property fails when `X` is one of `'(A . T)` (whatever that is), `NIL`, or `'(( - #\A) (-2 0))` (whatever **that** is). We've seen this before. The problem is that we wrote down the property without constraining `X` to have the right expected input values. We said

    (<= (expt 10 (natlog x)) 
        x)

But what we really meant was

    (implies (natp x)
             (<= (expt 10 (natlog x)) 
                 x))

Let's fix those properties.

In [ ]:
(defsnapshot natlog-definition)

(definec natlog (x :nat) :nat
  (if (or (< x 10)
          (not (and (integerp x)
                    (> x 0))))
      0
      (+ (natlog (floor x 10)) 1)))

(check-expect (natlog 1) 0)
(check-expect (natlog 10) 1)
(check-expect (natlog 100) 2)
(check-expect (natlog 1000) 3)
(check-expect (natlog 98) 1)
(check-expect (natlog 102) 2)
(check-expect (natlog 980) 2)
(check-expect (natlog 1020) 3)

(test? (implies (natp x)
                (<= (expt 10 (natlog x)) 
                    x)))
(test? (implies (and (natp x)
                     (> n (natlog x)))
                (> (expt 10 n) x)))

Well, we're getting closer. This time, ACL2 reports that the property succeeds when `n` is 5, 9, or 485. But it fails when `n=0`. Let's see what happens then.

We know that `(natlog 0) = 0`. That does make sense. But $2^0 = 1 > 0$. That's unfortunate. What the function is returning seems right. But the property is not. $2^n \ge 1$ for any natural number $n$, so we really can't expect to return a "corrrect" value for `(natlog 0)`. In fact, at this point you may be thinking that $\log 0$ is undefined, right? Our function returns 0, because it has to return **something** for all inputs. But maybe 0 isn't a valid input. The valid inputs are **not** the natural numbers. Only positive integers should be allowed!

So that suggests we should our funciton definition from expecting `x :nat` to expecting `x :pos`. Recall that `:pos` denotes the type of positive integers. The output of the function is still `:nat`, since `(natlog 1) = 0`. We should also change our properties, since `x` needs to be a positive integer, as opposed to a natural number.

OK, here's the final(?) version:

In [ ]:
(defsnapshot natlog-definition)

(definec natlog (x :pos) :nat
  (if (or (< x 10)
          (not (and (integerp x)
                    (> x 0))))
      0
      (+ (natlog (floor x 10)) 1)))

(check-expect (natlog 1) 0)
(check-expect (natlog 10) 1)
(check-expect (natlog 100) 2)
(check-expect (natlog 1000) 3)
(check-expect (natlog 98) 1)
(check-expect (natlog 102) 2)
(check-expect (natlog 980) 2)
(check-expect (natlog 1020) 3)

(test? (implies (posp x)
                (<= (expt 10 (natlog x)) 
                    x)))
(test? (implies (and (posp x)
                     (> n (natlog x)))
                (> (expt 10 n) x)))

Progress! The first property passes, as in ACL2 does not find any counterexamples. The second property didn't do as well. It did pass sometimes, for example when `n=11` and `x=196`. But the property failed when `n=232/11` and `x=92`, and when `n=#c(1 2)` and `x=41`. Ah yes, we've seen this before. The property correctly says that `x` should be a positive integer, but it doesn't constrain `n` at all. This should be easy to fix. What we meant was

    (implies (and (posp x)
                  (natp n)
                  (> n (natlog x)))
             (> (expt 10 n) x))

Here we go (again)!

In [ ]:
(defsnapshot natlog-definition)

(definec natlog (x :pos) :nat
  (if (or (< x 10)
          (not (and (integerp x)
                    (> x 0))))
      0
      (+ (natlog (floor x 10)) 1)))

(check-expect (natlog 1) 0)
(check-expect (natlog 10) 1)
(check-expect (natlog 100) 2)
(check-expect (natlog 1000) 3)
(check-expect (natlog 98) 1)
(check-expect (natlog 102) 2)
(check-expect (natlog 980) 2)
(check-expect (natlog 1020) 3)

(test? (implies (posp x)
                (<= (expt 10 (natlog x)) 
                    x)))
(test? (implies (and (posp x)
                     (natp n)
                     (> n (natlog x)))
                (> (expt 10 n) x)))

Sweet, sweet success! This is it. ACL2 checked both of our properties by testing them with 1,000s of possible inputs, and the property was true for each of those inputs! Finally. Now we can rest.

> ***Coda:*** After the stunning success of randomized testing, you may be tempted to try to use `thm` instead of `test?` to see if ACL2 can prove this conjecture and remove all doubts. Alas! ACL2 fails to prove this. The property is really true, but the proof is tricky enough that ACL2 cannot discover it for itself. But that's OK. This tutorial is about using TDD to define equations properly, and that's what we did. Learning how to prove properties is for another tutorial (and in any case, is something better suited for the second semester of your journey with ACL2.)